In [3]:
import pandas as pd
from pandas import DataFrame
from datetime import datetime

In [4]:
dataset = "./dataset/metadata-norm.csv"

In [5]:
df = pd.read_csv(dataset)

In [6]:
df.head()

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
0,discharge,2010-07-21 15:00:35,4,B0047,0,1,00001.csv,1.6743047446975208,NaN,NaN
1,impedance,2010-07-21 16:53:45,24,B0047,1,2,00002.csv,NaN,0.05605783343888099,0.20097016584458333
2,charge,2010-07-21 17:25:40,4,B0047,2,3,00003.csv,NaN,NaN,NaN
3,impedance,2010-07-21 20:31:05,24,B0047,3,4,00004.csv,NaN,0.05319185850921101,0.16473399914864734
4,discharge,2010-07-21 21:02:56,4,B0047,4,5,00005.csv,1.5243662105099023,NaN,NaN


In [7]:
print(f"Unique Battery Types: {len(df["battery_id"].unique())}")

Unique Battery Types: 34


In [8]:
print([id for id in df["battery_id"].unique()])

['B0047', 'B0045', 'B0048', 'B0046', 'B0043', 'B0032', 'B0039', 'B0040', 'B0029', 'B0028', 'B0042', 'B0034', 'B0038', 'B0033', 'B0030', 'B0041', 'B0027', 'B0044', 'B0036', 'B0025', 'B0026', 'B0031', 'B0049', 'B0050', 'B0052', 'B0051', 'B0006', 'B0005', 'B0007', 'B0018', 'B0053', 'B0054', 'B0056', 'B0055']


In [80]:
def sort_by_time(d):
    return d.sort_values("start_time")

def get_battery_info_by_filename(filename):
    return pd.read_csv(f"./dataset/data/{filename}")["Battery_impedance"]

def parse_complex_number(s):
    s = complex(s)
    return str(f"{s.real},{s.imag}")

def get_rows_by_battery_id(battery_id, type="re",freq="0.1"):
    d = df[df['battery_id'] == battery_id]
    d[d["type"] == "impedance"]
    
    if type == "re" or type == "rct":
        if type == "re":
            d = d[d["type"] == "impedance"]
            d = d.drop(columns=['Rct', 'ambient_temperature', 'test_id', 'uid', 'filename', 'Capacity','type', 'battery_id'])
            d = sort_by_time(d)
            return d
        else:    
            d = d[d["type"] == "impedance"]
            d = d.drop(columns=['Re', 'ambient_temperature', 'test_id', 'uid', 'filename', 'Capacity','type', 'battery_id'])
            d = sort_by_time(d)
            return d
    else:
        d = d[d["type"] == "impedance"]
        d = d.drop(columns=['Rct', 'ambient_temperature', 'test_id', 'uid', 'Re', 'Capacity','type', 'battery_id'])
        d = d.join(d["filename"].apply(get_battery_info_by_filename))
        d.drop(columns=["filename"], inplace=True)
        d = sort_by_time(d)
        idx = str(int((float(freq) * 10)))
        d = d.iloc[:, [0, int(idx)]]
        d["Re"] = d[d.columns[1]].apply(lambda x: float(parse_complex_number(x).split(',')[0]))
        d["Im"] = d[d.columns[1]].apply(lambda x: float(parse_complex_number(x).split(',')[1]))
        d.drop(columns=[d.columns[1]], inplace=True)
        return d["Re"].to_list(), d["Im"].to_list()

In [83]:
get_rows_by_battery_id("B0050", "re", "0.48").head()

,start_time,Re
4319,2010-08-23 18:25:01,0.08458047397458193
4321,2010-08-23 22:01:52,0.07617570199996643
4329,2010-08-26 12:25:47,-968924452345684.0
4331,2010-08-26 16:02:42,-4986.050426740892
4341,2010-08-27 09:22:52,0.10727800454395818
